In [22]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_columns', None)

In [23]:
df = pd.read_csv("2024.csv")

In [24]:
df.isnull().sum()

YEAR                        0
MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
MKT_CARRIER                 0
OP_CARRIER                  0
TAIL_NUM                20966
ORIGIN                      0
ORIGIN_CITY_NAME            0
ORIGIN_STATE_NM             0
DEST                        0
DEST_CITY_NAME              0
DEST_STATE_NM               0
CRS_DEP_TIME                0
DEP_TIME                85763
DEP_DELAY               86020
TAXI_OUT                88729
WHEELS_OFF              88729
WHEELS_ON               90421
TAXI_IN                 90421
CRS_ARR_TIME                0
ARR_TIME                90419
ARR_DELAY              104635
CANCELLED                   0
DIVERTED                    0
CRS_ELAPSED_TIME            1
ACTUAL_ELAPSED_TIME    104635
AIR_TIME               104635
DISTANCE                    0
dtype: int64

In [25]:
#print(df['CANCELLED'].value_counts()[1.0])

In [26]:
missing_data = pd.DataFrame({
    'Kolon': df.columns,
    'Bos_Sayisi': df.isnull().sum(),
    'Bos_Yuzdesi': (df.isnull().sum() / len(df)) * 100
})
print(missing_data)

                                   Kolon  Bos_Sayisi  Bos_Yuzdesi
YEAR                                YEAR           0     0.000000
MONTH                              MONTH           0     0.000000
DAY_OF_MONTH                DAY_OF_MONTH           0     0.000000
DAY_OF_WEEK                  DAY_OF_WEEK           0     0.000000
MKT_CARRIER                  MKT_CARRIER           0     0.000000
OP_CARRIER                    OP_CARRIER           0     0.000000
TAIL_NUM                        TAIL_NUM       20966     0.371465
ORIGIN                            ORIGIN           0     0.000000
ORIGIN_CITY_NAME        ORIGIN_CITY_NAME           0     0.000000
ORIGIN_STATE_NM          ORIGIN_STATE_NM           0     0.000000
DEST                                DEST           0     0.000000
DEST_CITY_NAME            DEST_CITY_NAME           0     0.000000
DEST_STATE_NM              DEST_STATE_NM           0     0.000000
CRS_DEP_TIME                CRS_DEP_TIME           0     0.000000
DEP_TIME  

In [27]:
# Boş verileri df den çıkarmak için iptal edilen ve aktarılan uçuşları çıkarıyoruz.
df = df[df['CANCELLED'] != 1.0]
df = df[df['DIVERTED'] != 1.0]

# İptal edilen ve aktarılan uçuşlar için gerekli olmayan sütunları siliyoruz.
df.drop(['CANCELLED', 'DIVERTED'], axis=1, inplace=True)

In [28]:
df["DEST_CITY_NAME"] = df["DEST_CITY_NAME"].str.split(",").str[0]
df["ORIGIN_CITY_NAME"] = df["ORIGIN_CITY_NAME"].str.split(",").str[0]

In [29]:
# Float veri tipindeki kolonları int'e dönüştür
df = df.astype({col: 'int64' for col in df.select_dtypes(include='float').columns})

In [30]:
# DAY_OF_MONTH kolonunun adını DAY olarak değiştir
df.rename(columns={'DAY_OF_MONTH': 'DAY'}, inplace=True)

# Tarih oluşturma
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# YEAR, MONTH, DAY kolonlarını sil
df.drop(['YEAR', 'MONTH', 'DAY'], axis=1, inplace=True)

In [9]:
"""import pandas as pd

# Saat formatındaki sütunları (HHMM) datetime formatına çevirelim
def convert_to_datetime(df, column):
    df[column] = df[column].astype(str).str.zfill(4)  # Eksik sıfırları tamamla (örn: "530" -> "0530")

    # "2400" olanları "0000" yap
    df[column] = df[column].replace("2400", "0000")

    # Datetime formatına çevir
    df[column] = pd.to_datetime(
        df["DATE"].astype(str) + " " +
        df[column].str[:2] + ":" + df[column].str[2:],
        format="%Y-%m-%d %H:%M"
    )
    return df

# Dönüştürülecek tüm zaman sütunlarını düzenleyelim
time_columns = ["CRS_DEP_TIME", "DEP_TIME", "WHEELS_OFF", "WHEELS_ON", "CRS_ARR_TIME", "ARR_TIME"]
for col in time_columns:
    df = convert_to_datetime(df, col)
"""

In [31]:
columns_to_convert = ['CRS_DEP_TIME', 'DEP_TIME', 'WHEELS_OFF', 'WHEELS_ON', 'CRS_ARR_TIME', 'ARR_TIME']

# Her bir kolon için saat formatına çevirme
for col in columns_to_convert:
    # 4 haneli olacak şekilde doldur
    df[col] = df[col].apply(lambda x: f"{int(x):04d}")
    
    # 2400 değerini 0000 olarak değiştir
    df[col] = df[col].replace('2400', '0000')
"""    
    # Saat formatına çevir
    df[col] = pd.to_datetime(df[col], format='%H%M', errors='coerce').dt.time
"""

"    \n    # Saat formatına çevir\n    df[col] = pd.to_datetime(df[col], format='%H%M', errors='coerce').dt.time\n"

In [32]:
#!pip install airportsdata pytz

   ---------------------------------------- 0.0/912.7 kB ? eta -:--:--
   ---------------------------------------- 10.2/912.7 kB ? eta -:--:--
   -- ------------------------------------ 61.4/912.7 kB 812.7 kB/s eta 0:00:02
   -------- ------------------------------- 184.3/912.7 kB 1.6 MB/s eta 0:00:01
   ----------- ---------------------------- 266.2/912.7 kB 1.6 MB/s eta 0:00:01
   -------------- ------------------------- 327.7/912.7 kB 1.6 MB/s eta 0:00:01
   -------------------- ------------------- 471.0/912.7 kB 1.8 MB/s eta 0:00:01
   --------------------------- ------------ 634.9/912.7 kB 2.1 MB/s eta 0:00:01
   ----------------------------------- ---- 798.7/912.7 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 912.7/912.7 kB 2.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
from airportsdata import load
import pytz
from datetime import datetime, timedelta

# Havaalanı verilerini yükle
airports = load('IATA')

# Havaalanı koduna göre saat dilimi getiren fonksiyon
def get_timezone(iata_code):
    airport = airports.get(iata_code, {})  # Havaalanı bilgilerini al, yoksa boş sözlük döndür
    return airport.get('tz', None)  # 'tz' anahtarını kullanarak saat dilimini al



In [35]:
import airportsdata

# Havaalanı verilerini yükleyelim
airports = airportsdata.load('IATA')

# Örnek birkaç havaalanının verisini yazdıralım
sample_airports = list(airports.keys())[:5]  # İlk 5 havaalanı kodunu al
for airport_code in sample_airports:
    print(airport_code, ":", airports[airport_code])  # Havaalanı bilgilerini yazdır


OCA : {'icao': '07FA', 'iata': 'OCA', 'name': 'Ocean Reef Club Airport', 'city': 'Key Largo', 'subd': 'Florida', 'country': 'US', 'elevation': 6.0, 'lat': 25.32432, 'lon': -80.27573, 'tz': 'America/New_York', 'lid': '07FA'}
CYT : {'icao': '0AA1', 'iata': 'CYT', 'name': 'Yakataga Airport', 'city': 'Yakataga', 'subd': 'Alaska', 'country': 'US', 'elevation': 16.0, 'lat': 60.08085, 'lon': -142.49549, 'tz': 'America/Anchorage', 'lid': '0AA1'}
FWL : {'icao': '0AA4', 'iata': 'FWL', 'name': 'Farewell Airport', 'city': 'Farewell', 'subd': 'Alaska', 'country': 'US', 'elevation': 1535.0, 'lat': 62.50918, 'lon': -153.89063, 'tz': 'America/Anchorage', 'lid': '0AA4'}
CSE : {'icao': '0CO2', 'iata': 'CSE', 'name': 'Crested Butte Airpark', 'city': 'Crested Butte', 'subd': 'Colorado', 'country': 'US', 'elevation': 8980.0, 'lat': 38.85194, 'lon': -106.93282, 'tz': 'America/Denver', 'lid': '0CO2'}
CUS : {'icao': '0NM0', 'iata': 'CUS', 'name': 'Columbus Municipal Airport', 'city': 'Columbus', 'subd': 'New 

In [37]:
df['ORIGIN_TZ'] = df['ORIGIN'].apply(get_timezone)
df['DEST_TZ'] = df['DEST'].apply(get_timezone)


In [38]:
def convert_to_utc(time_str, date, timezone_str):
    if pd.isna(time_str) or pd.isna(timezone_str):
        return None
    # HHMM formatındaki saati integer olarak al
    hour = int(time_str[:2])
    minute = int(time_str[2:])
    # Tarih ve saati birleştirerek datetime objesi oluştur
    local_time = datetime.combine(date, datetime.min.time()) + timedelta(hours=hour, minutes=minute)
    # Yerel saat dilimini ayarla
    local_tz = pytz.timezone(timezone_str)
    local_dt = local_tz.localize(local_time)
    # UTC'ye çevir
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt

In [39]:
# ORIGIN bazlı sütunları UTC'ye çevir
origin_time_columns = ["DEP_TIME", "CRS_DEP_TIME", "WHEELS_OFF"]
for col in origin_time_columns:
    df[f"{col}_UTC"] = df.apply(lambda row: convert_to_utc(row[col], row["DATE"], row["ORIGIN_TZ"]), axis=1)

# DEST bazlı sütunları UTC'ye çevir
dest_time_columns = ["ARR_TIME", "CRS_ARR_TIME", "WHEELS_ON"]
for col in dest_time_columns:
    df[f"{col}_UTC"] = df.apply(lambda row: convert_to_utc(row[col], row["DATE"], row["DEST_TZ"]), axis=1)


In [40]:
df[["ORIGIN", "DEP_TIME", "DEP_TIME_UTC", "ORIGIN_TZ",
    "DEST", "ARR_TIME", "ARR_TIME_UTC", "DEST_TZ"]].sample(10)


,ORIGIN,DEP_TIME,DEP_TIME_UTC,ORIGIN_TZ,DEST,ARR_TIME,ARR_TIME_UTC,DEST_TZ
4641559,LGB,1847,2024-08-14 01:47:00+00:00,America/Los_Angeles,OAK,2003,2024-08-14 03:03:00+00:00,America/Los_Angeles
2480821,SEA,0047,2024-05-05 07:47:00+00:00,America/Los_Angeles,IAH,0650,2024-05-05 11:50:00+00:00,America/Chicago
717023,CLT,2232,2024-02-09 03:32:00+00:00,America/New_York,IAD,2357,2024-02-09 04:57:00+00:00,America/New_York
4556884,OAK,1246,2024-08-09 19:46:00+00:00,America/Los_Angeles,DEN,1611,2024-08-09 22:11:00+00:00,America/Denver
3621416,BDL,0958,2024-06-28 13:58:00+00:00,America/New_York,CLT,1151,2024-06-28 15:51:00+00:00,America/New_York
3923143,MHK,0535,2024-07-12 10:35:00+00:00,America/Chicago,DFW,0702,2024-07-12 12:02:00+00:00,America/Chicago
4620909,SMF,1253,2024-08-12 19:53:00+00:00,America/Los_Angeles,SNA,1410,2024-08-12 21:10:00+00:00,America/Los_Angeles
3035378,LGA,0958,2024-06-01 13:58:00+00:00,America/New_York,ORD,1136,2024-06-01 16:36:00+00:00,America/Chicago
140252,OGG,2316,2024-01-09 09:16:00+00:00,Pacific/Honolulu,SEA,0650,2024-01-08 14:50:00+00:00,America/Los_Angeles
2603839,ISP,1929,2024-05-11 23:29:00+00:00,America/New_York,TPA,2219,2024-05-12 02:19:00+00:00,America/New_York


In [41]:
# Kategorik olması gereken sütunları dönüştürelim
cat_columns = ["MKT_CARRIER", "OP_CARRIER", "TAIL_NUM", "ORIGIN", "ORIGIN_CITY_NAME",
               "ORIGIN_STATE_NM", "DEST", "DEST_CITY_NAME", "DEST_STATE_NM"]

for col in cat_columns:
    df[col] = df[col].astype("category")


In [42]:
df.head()

,DAY_OF_WEEK,MKT_CARRIER,OP_CARRIER,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DATE,ORIGIN_TZ,DEST_TZ,DEP_TIME_UTC,CRS_DEP_TIME_UTC,WHEELS_OFF_UTC,ARR_TIME_UTC,CRS_ARR_TIME_UTC,WHEELS_ON_UTC
0,1,AA,AA,N656AW,MSN,Madison,Wisconsin,CLT,Charlotte,North Carolina,0605,0600,-5,24,0624,0901,10,0916,0911,-5,131,131,97,708,2024-01-01,America/Chicago,America/New_York,2024-01-01 12:00:00+00:00,2024-01-01 12:05:00+00:00,2024-01-01 12:24:00+00:00,2024-01-01 14:11:00+00:00,2024-01-01 14:16:00+00:00,2024-01-01 14:01:00+00:00
1,1,AA,AA,N912UY,CLT,Charlotte,North Carolina,STL,St. Louis,Missouri,2255,2306,11,12,2318,2340,19,2359,2359,0,124,113,82,575,2024-01-01,America/New_York,America/Chicago,2024-01-02 04:06:00+00:00,2024-01-02 03:55:00+00:00,2024-01-02 04:18:00+00:00,2024-01-02 05:59:00+00:00,2024-01-02 05:59:00+00:00,2024-01-02 05:40:00+00:00
2,1,AA,AA,N111US,PHL,Philadelphia,Pennsylvania,CLT,Charlotte,North Carolina,1737,1901,84,31,1932,2045,7,1929,2052,83,112,111,73,449,2024-01-01,America/New_York,America/New_York,2024-01-02 00:01:00+00:00,2024-01-01 22:37:00+00:00,2024-01-02 00:32:00+00:00,2024-01-02 01:52:00+00:00,2024-01-02 00:29:00+00:00,2024-01-02 01:45:00+00:00
3,1,AA,AA,N854NN,DFW,Dallas/Fort Worth,Texas,STL,St. Louis,Missouri,2109,2134,25,12,2146,2259,6,2253,2305,12,104,91,73,550,2024-01-01,America/Chicago,America/Chicago,2024-01-02 03:34:00+00:00,2024-01-02 03:09:00+00:00,2024-01-02 03:46:00+00:00,2024-01-02 05:05:00+00:00,2024-01-02 04:53:00+00:00,2024-01-02 04:59:00+00:00
4,1,AA,AA,N449AN,LAS,Las Vegas,Nevada,CLT,Charlotte,North Carolina,0800,0755,-5,18,0813,1451,8,1513,1459,-14,253,244,218,1916,2024-01-01,America/Los_Angeles,America/New_York,2024-01-01 15:55:00+00:00,2024-01-01 16:00:00+00:00,2024-01-01 16:13:00+00:00,2024-01-01 19:59:00+00:00,2024-01-01 20:13:00+00:00,2024-01-01 19:51:00+00:00


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5539509 entries, 0 to 5644143
Data columns (total 33 columns):
 #   Column               Dtype              
---  ------               -----              
 0   DAY_OF_WEEK          int64              
 1   MKT_CARRIER          category           
 2   OP_CARRIER           category           
 3   TAIL_NUM             category           
 4   ORIGIN               category           
 5   ORIGIN_CITY_NAME     category           
 6   ORIGIN_STATE_NM      category           
 7   DEST                 category           
 8   DEST_CITY_NAME       category           
 9   DEST_STATE_NM        category           
 10  CRS_DEP_TIME         object             
 11  DEP_TIME             object             
 12  DEP_DELAY            int64              
 13  TAXI_OUT             int64              
 14  WHEELS_OFF           object             
 15  WHEELS_ON            object             
 16  TAXI_IN              int64              
 17  CRS_ARR_TIME 

In [21]:
import numpy as np

# NumPy'nin tüm verileri göstermesini sağla
np.set_printoptions(threshold=np.inf)

# Benzersiz değerleri al ve sırala
unique_values = df['ORIGIN'].unique()
sorted_values = np.sort(unique_values)

# Benzersiz değer sayısını yazdır
print(f"Toplam benzersiz değer sayısı: {len(unique_values)}")

# Sıralanmış benzersiz değerleri yazdır
print("Sıralanmış benzersiz değerler:")
print(sorted_values)


Toplam benzersiz değer sayısı: 356
Sıralanmış benzersiz değerler:
['ABE' 'ABI' 'ABQ' 'ABR' 'ABY' 'ACK' 'ACT' 'ACV' 'ACY' 'ADK' 'ADQ' 'AEX'
 'AGS' 'AKN' 'ALB' 'ALO' 'ALW' 'AMA' 'ANC' 'APN' 'ART' 'ASE' 'ATL' 'ATW'
 'AUS' 'AVL' 'AVP' 'AZA' 'AZO' 'BDL' 'BET' 'BFF' 'BFL' 'BGM' 'BGR' 'BHM'
 'BIH' 'BIL' 'BIS' 'BJI' 'BLI' 'BLV' 'BMI' 'BNA' 'BOI' 'BOS' 'BPT' 'BQK'
 'BQN' 'BRD' 'BRO' 'BRW' 'BTM' 'BTR' 'BTV' 'BUF' 'BUR' 'BWI' 'BZN' 'CAE'
 'CAK' 'CDC' 'CDV' 'CHA' 'CHO' 'CHS' 'CID' 'CIU' 'CKB' 'CLE' 'CLL' 'CLT'
 'CMH' 'CMI' 'CMX' 'CNY' 'COD' 'COS' 'COU' 'CPR' 'CRP' 'CRW' 'CSG' 'CVG'
 'CWA' 'CYS' 'DAB' 'DAL' 'DAY' 'DCA' 'DDC' 'DEC' 'DEN' 'DFW' 'DHN' 'DIK'
 'DLG' 'DLH' 'DRO' 'DSM' 'DTW' 'DVL' 'EAT' 'ECP' 'EGE' 'EKO' 'ELM' 'ELP'
 'ERI' 'ESC' 'EUG' 'EVV' 'EWN' 'EWR' 'EYW' 'FAI' 'FAR' 'FAT' 'FAY' 'FCA'
 'FLG' 'FLL' 'FLO' 'FNT' 'FOD' 'FSD' 'FSM' 'FWA' 'GCC' 'GCK' 'GEG' 'GFK'
 'GGG' 'GJT' 'GNV' 'GPT' 'GRB' 'GRI' 'GRK' 'GRR' 'GSO' 'GSP' 'GST' 'GTF'
 'GTR' 'GUC' 'GUM' 'HDN' 'HGR' 'HHH' 'HIB' 'HLN' 'HNL' 'HO

In [44]:
# "ACTUAL_ELAPSED_TIME" kolonu değeri 11 olan satırları filtreleyin
filtered_df = df[df['DISTANCE'] == 5095]

# Filtrelenmiş satırları ekrana yazdırın
filtered_df

,DAY_OF_WEEK,MKT_CARRIER,OP_CARRIER,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DATE,ORIGIN_TZ,DEST_TZ,DEP_TIME_UTC,CRS_DEP_TIME_UTC,WHEELS_OFF_UTC,ARR_TIME_UTC,CRS_ARR_TIME_UTC,WHEELS_ON_UTC
10240,1,HA,HA,N391HA,BOS,Boston,Massachusetts,HNL,Honolulu,Hawaii,0800,0807,7,12,0819,1355,8,1425,1403,-22,685,656,636,5095,2024-01-01,America/New_York,Pacific/Honolulu,2024-01-01 13:07:00+00:00,2024-01-01 13:00:00+00:00,2024-01-01 13:19:00+00:00,2024-01-02 00:03:00+00:00,2024-01-02 00:25:00+00:00,2024-01-01 23:55:00+00:00
10242,1,HA,HA,N361HA,HNL,Honolulu,Hawaii,BOS,Boston,Massachusetts,1515,1525,10,12,1537,0543,6,0600,0549,-11,585,564,546,5095,2024-01-01,Pacific/Honolulu,America/New_York,2024-01-02 01:25:00+00:00,2024-01-02 01:15:00+00:00,2024-01-02 01:37:00+00:00,2024-01-01 10:49:00+00:00,2024-01-01 11:00:00+00:00,2024-01-01 10:43:00+00:00
29936,2,HA,HA,N361HA,BOS,Boston,Massachusetts,HNL,Honolulu,Hawaii,0800,0804,4,15,0819,1349,40,1425,1429,4,685,685,630,5095,2024-01-02,America/New_York,Pacific/Honolulu,2024-01-02 13:04:00+00:00,2024-01-02 13:00:00+00:00,2024-01-02 13:19:00+00:00,2024-01-03 00:29:00+00:00,2024-01-03 00:25:00+00:00,2024-01-02 23:49:00+00:00
49811,3,HA,HA,N393HA,HNL,Honolulu,Hawaii,BOS,Boston,Massachusetts,1515,1515,0,12,1527,0538,4,0600,0542,-18,585,567,551,5095,2024-01-03,Pacific/Honolulu,America/New_York,2024-01-04 01:15:00+00:00,2024-01-04 01:15:00+00:00,2024-01-04 01:27:00+00:00,2024-01-03 10:42:00+00:00,2024-01-03 11:00:00+00:00,2024-01-03 10:38:00+00:00
68989,4,HA,HA,N393HA,BOS,Boston,Massachusetts,HNL,Honolulu,Hawaii,0800,0754,-6,20,0814,1355,8,1425,1403,-22,685,669,641,5095,2024-01-04,America/New_York,Pacific/Honolulu,2024-01-04 12:54:00+00:00,2024-01-04 13:00:00+00:00,2024-01-04 13:14:00+00:00,2024-01-05 00:03:00+00:00,2024-01-05 00:25:00+00:00,2024-01-04 23:55:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572334,5,HA,HA,N375HA,HNL,Honolulu,Hawaii,BOS,Boston,Massachusetts,1415,1414,-1,18,1432,0550,9,0600,0559,-1,585,585,558,5095,2024-09-27,Pacific/Honolulu,America/New_York,2024-09-28 00:14:00+00:00,2024-09-28 00:15:00+00:00,2024-09-28 00:32:00+00:00,2024-09-27 09:59:00+00:00,2024-09-27 10:00:00+00:00,2024-09-27 09:50:00+00:00
5591980,6,HA,HA,N375HA,BOS,Boston,Massachusetts,HNL,Honolulu,Hawaii,0800,0803,3,39,0842,1247,40,1310,1327,17,670,684,605,5095,2024-09-28,America/New_York,Pacific/Honolulu,2024-09-28 12:03:00+00:00,2024-09-28 12:00:00+00:00,2024-09-28 12:42:00+00:00,2024-09-28 23:27:00+00:00,2024-09-28 23:10:00+00:00,2024-09-28 22:47:00+00:00
5612847,7,HA,HA,N384HA,HNL,Honolulu,Hawaii,BOS,Boston,Massachusetts,1635,1634,-1,15,1649,0826,11,0815,0837,22,580,603,577,5095,2024-09-29,Pacific/Honolulu,America/New_York,2024-09-30 02:34:00+00:00,2024-09-30 02:35:00+00:00,2024-09-30 02:49:00+00:00,2024-09-29 12:37:00+00:00,2024-09-29 12:15:00+00:00,2024-09-29 12:26:00+00:00
5635232,1,HA,HA,N384HA,BOS,Boston,Massachusetts,HNL,Honolulu,Hawaii,1015,1026,11,18,1044,1436,31,1520,1507,-13,665,641,592,5095,2024-09-30,America/New_York,Pacific/Honolulu,2024-09-30 14:26:00+00:00,2024-09-30 14:15:00+00:00,2024-09-30 14:44:00+00:00,2024-10-01 01:07:00+00:00,2024-10-01 01:20:00+00:00,2024-10-01 00:36:00+00:00


In [9]:
#print(df.columns)

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'MKT_CARRIER',
       'OP_CARRIER', 'TAIL_NUM', 'ORIGIN', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_NM', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_NM',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME',
       'AIR_TIME', 'DISTANCE'],
      dtype='object')


In [45]:
# Sadece int kolonları seçip describe'ı gösterme
int_columns_describe = df.select_dtypes(include='int').describe()
int_columns_describe

,DAY_OF_WEEK,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
count,5.539509e+06,5.539509e+06,5.539509e+06,5.539509e+06,5.539509e+06,5.539509e+06,5.539509e+06,5.539509e+06,5.539509e+06
mean,3.978397e+00,1.383937e+01,1.801424e+01,8.357802e+00,8.743955e+00,1.436213e+02,1.385261e+02,1.121541e+02,8.082412e+02
std,2.008406e+00,5.863871e+01,9.878335e+00,6.955433e+00,6.081433e+01,7.157618e+01,7.164560e+01,6.976224e+01,5.914965e+02
min,1.000000e+00,-9.600000e+01,1.000000e+00,1.000000e+00,-1.170000e+02,6.000000e+00,1.500000e+01,5.000000e+00,1.100000e+01
25%,2.000000e+00,-6.000000e+00,1.200000e+01,5.000000e+00,-1.500000e+01,9.100000e+01,8.600000e+01,6.100000e+01,3.740000e+02
50%,4.000000e+00,-2.000000e+00,1.500000e+01,6.000000e+00,-5.000000e+00,1.260000e+02,1.220000e+02,9.500000e+01,6.510000e+02
75%,6.000000e+00,1.000000e+01,2.100000e+01,1.000000e+01,1.100000e+01,1.740000e+02,1.690000e+02,1.420000e+02,1.046000e+03
max,7.000000e+00,3.777000e+03,2.140000e+02,4.440000e+02,3.803000e+03,8.590000e+02,1.354000e+03,1.338000e+03,5.095000e+03


In [46]:
df.duplicated().sum()

0

In [ ]:
df.to_csv("2024_V1.csv", index=False)

print("İşlem tamamlandı! csv dosyası oluşturuldu.")

İşlem tamamlandı! csv dosyası oluşturuldu.
